In [21]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [22]:
load_dotenv(override=True)

GEMINI_MODEL = 'gemini-2.5-flash'
GEMINI_BASE_URL = 'https://generativelanguage.googleapis.com/v1beta/openai/'
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY') 

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=GEMINI_API_KEY)

In [23]:
links = fetch_website_links("https://mtsortiz.vercel.app/")
links

['https://www.linkedin.com/in/matiasnicolasortiz/',
 'https://github.com/mtsortiz',
 'https://linkedin.com/in/matiasnicolasortiz/',
 'https://github.com/mtsortiz',
 'mailto:matiasnortiz.dev@gmail.com',
 'https://drive.google.com/file/d/19uD8ttoRBcRMhvmixjq00HgQ1-MYTXl1/view?usp=sharing']

In [24]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [25]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [26]:
print(get_links_user_prompt("https://mtsortiz.vercel.app/"))


Here is the list of links on the website https://mtsortiz.vercel.app/ -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://www.linkedin.com/in/matiasnicolasortiz/
https://github.com/mtsortiz
https://linkedin.com/in/matiasnicolasortiz/
https://github.com/mtsortiz
mailto:matiasnortiz.dev@gmail.com
https://drive.google.com/file/d/19uD8ttoRBcRMhvmixjq00HgQ1-MYTXl1/view?usp=sharing


In [27]:
def select_relevant_links(url):
    response = gemini.chat.completions.create(
        model=GEMINI_MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [28]:
select_relevant_links("https://mtsortiz.vercel.app/")

{'links': [{'type': 'about page',
   'url': 'https://www.linkedin.com/in/matiasnicolasortiz/'},
  {'type': 'careers page',
   'url': 'https://drive.google.com/file/d/19uD8ttoRBcRMhvmixjq00HgQ1-MYTXl1/view?usp=sharing'}]}

In [29]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {GEMINI_MODEL}")
    response = gemini.chat.completions.create(
        model=GEMINI_MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [30]:
select_relevant_links("https://mtsortiz.vercel.app/")

Selecting relevant links for https://mtsortiz.vercel.app/ by calling gemini-2.5-flash
Found 3 relevant links


{'links': [{'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/matiasnicolasortiz/'},
  {'type': 'portfolio page', 'url': 'https://github.com/mtsortiz'},
  {'type': 'resume',
   'url': 'https://drive.google.com/file/d/19uD8ttoRBcRMhvmixjq00HgQ1-MYTXl1/view?usp=sharing'}]}

In [31]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash
Found 16 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'models overview page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets overview page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces overview page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://huggingface.co/join'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'brand assets page', 'url': 'https://huggingface.co/brand'},
  {'type': 'changelog page', 'url': 'https://huggingface.co/changelog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'learning page', 'url': 'https://huggingface.co/learn'},
  {'type': 'blog page', 'url': 'https://huggingface.co/bl

In [32]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        try:
            result += f"\n\n### Link: {link['type']}\n"
            result += fetch_website_contents(link["url"])
        except Exception as e:
            print(f"⚠️ Could not fetch {link['url']}: {str(e)[:100]}")
            result += f"\n\n(Could not fetch this link)\n"
    return result

In [33]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash
Found 16 relevant links
⚠️ Could not fetch https://status.huggingface.co/: HTTPSConnectionPool(host='status.huggingface.co', port=443): Max retries exceeded with url: / (Cause
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2.1
Updated
6 days ago
•
171k
•
773
tencent/HY-MT1.5-1.8B
Updated
1 day ago
•
1.96k
•
474
zai-org/GLM-4.7
Updated
10 days ago
•
31.2k
•
1.39k
Qwen/Qwen-Image-2512
Updated
2 days ago
•
5.83k
•
310
LGAI-EXAONE/K-EXAONE-236B-A23B
Updated
about 3 hours ago
•
250
•
272
Browse 2M+ models
Spaces
Running
Featured
3.36k
Wan2.2 Animate
👁
3.36k
Wan2.2 Animate
Running
on


In [34]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [35]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [36]:
get_brochure_user_prompt("MatiasOrtiz", "https://mtsortiz.vercel.app/")

Selecting relevant links for https://mtsortiz.vercel.app/ by calling gemini-2.5-flash
Found 3 relevant links


"\nYou are looking at a company called: MatiasOrtiz\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nMatías Nicolás Ortiz - Software Developer Portfolio\n\nHome\nAbout\nProjects\nContact\nHi, I'm Matías.\nSoftware Developer.\nBuilding efficient, scalable software with passion and precision.\nLinkedIn\nGitHub\nABOUT ME\nI'm a fourth-year Information Systems Engineering student with a passion for software development. Over the past two years, I've been diving deep into Java development, building personal projects with Spring Boot and exploring database design with MySQL. Currently expanding my horizons with Python and Kotlin, while maintaining a strong focus on clean code, design patterns, and software architecture best practices.\nPROJECTS\nAll\nPython\nJava\nKotlin\nSpring Boot\nMySQL\nHTML\nCSS\nConversational AI Agent: Virtual Waiter\nMusic Find

In [37]:
def create_brochure(company_name, url):
    response = gemini.chat.completions.create(
        model=GEMINI_MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [38]:
create_brochure("MatiasOrtiz", "https://mtsortiz.vercel.app/")

Selecting relevant links for https://mtsortiz.vercel.app/ by calling gemini-2.5-flash
Found 3 relevant links


**Matías Nicolás Ortiz - Crafting Tomorrow's Software, Today**

Welcome to the professional profile of Matías Nicolás Ortiz, a dedicated and passionate Software Developer poised to build the next generation of efficient and scalable software solutions. As a fourth-year Information Systems Engineering student, Matías combines a robust academic foundation with practical, hands-on experience, making him a valuable asset to any forward-thinking team.

**Who We Are (Matías Ortiz):**
Matías is driven by a profound passion for software development. Over the past two years, he has immersed himself in Java development, mastering frameworks like Spring Boot and database design with MySQL. Continuously expanding his technological horizons, he is actively delving into Python and Kotlin, ensuring a versatile skill set adaptable to diverse project requirements. His development philosophy centers on clean code, adherence to design patterns, and a solid understanding of software architecture best practices, guaranteeing maintainable and high-quality solutions.

**What We Offer (Matías Ortiz's Expertise):**
Matías's portfolio showcases a range of innovative projects, demonstrating his ability to tackle complex challenges and deliver tangible results. His work spans various domains and technologies, including:

*   **Conversational AI:** Development of an AI Agent (Virtual Waiter).
*   **Application Development:** Music Finder, Surveying App, Products Itinerary, ATM Simulator.
*   **Data Analysis:** Yoyo's Physical Data Analysis.
*   **Web Applications:** REST APIs, TODO's Web Application.
*   **Game Development:** Reversioned Candy Crush.

These projects highlight proficiency in Java, Python, Kotlin, Spring Boot, MySQL, HTML, and CSS, showcasing a comprehensive approach to software creation.

**Our Culture (Matías Ortiz's Approach):**
While this is a personal portfolio, Matías operates with a culture of continuous learning, precision, and a commitment to best practices. He thrives on building robust and maintainable software, always striving for excellence in code quality and architectural design. He is a proactive problem-solver with a strong drive for self-improvement and staying current with industry trends.

**Our Customers (Prospective Employers/Collaborators):**
Matías is actively seeking opportunities to contribute his skills to innovative projects and organizations. He is an ideal candidate for companies, startups, or collaborative teams looking for a highly motivated and skilled Software Developer who is passionate about building impactful software. His diverse project experience and strong foundation make him well-suited for roles in backend development, full-stack development, and application engineering.

**Career & Opportunities:**
Matías is in the process of building a dynamic career in software development. He is open to internship opportunities, junior developer roles, and collaborative projects where he can further apply his expertise and grow professionally. For those looking to hire a driven and skilled developer, Matías offers a blend of academic rigor and practical development experience.

**Let's Connect:**
Explore Matías's work and connect with him:

*   **Email:** matiasnortiz.dev@gmail.com
*   **LinkedIn:** [Available via landing page link]
*   **GitHub:** mtsortiz
*   **Download CV:** [Available via landing page link]

In [39]:
def stream_brochure(company_name, url):
    stream = gemini.chat.completions.create(
        model=GEMINI_MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
stream_brochure("MatiasOrtiz", "https://mtsortiz.vercel.app/")

Selecting relevant links for https://mtsortiz.vercel.app/ by calling gemini-2.5-flash
Found 3 relevant links


**Matías Nicolás Ortiz - Software Development**

Welcome to the world of Matías Nicolás Ortiz, a passionate and precise software developer dedicated to building efficient, scalable, and robust software solutions.

**About Us (About Matías)**
Matías is a fourth-year Information Systems Engineering student with a deep-seated passion for software development. His approach emphasizes clean code, adherence to design patterns, and robust software architecture best practices. He is committed to continuous learning, constantly expanding his technical horizons while honing his skills.

**Our Expertise & Technologies**
Matías brings over two years of hands-on experience in Java development, particularly with Spring Boot for backend services. He possesses strong skills in database design using MySQL. Currently, he is actively expanding his proficiency in Python and Kotlin, ensuring a versatile and modern development toolkit. His projects also demonstrate capabilities in HTML and CSS for web applications.

**Our Philosophy**
We believe that great software is built with both passion and precision. Matías strives to deliver solutions that are not only functional but also maintainable, scalable, and a pleasure to use. His dedication to best practices ensures high-quality outcomes for every project.

**Showcased Projects**
Matías's portfolio includes a diverse range of projects demonstrating his capabilities:
*   Conversational AI Agent: Virtual Waiter
*   Music Finder
*   Surveying App
*   ATM Simulator – Bank Employee
*   TODO's Web Application
*   Reversioned Candy Crush
*   REST APIs

**For Prospective Customers & Partners**
If you're looking for a dedicated and skilled software developer to bring your vision to life with efficient and scalable solutions, Matías Ortiz is eager to collaborate. His commitment to quality and modern development practices ensures valuable contributions to any project.

**For Investors & Recruiters**
Matías represents a promising talent in the software development landscape. As an Information Systems Engineering student with practical experience and a strong drive for continuous improvement, he is a valuable asset to any forward-thinking team. His portfolio showcases a practical application of theoretical knowledge and a clear understanding of industry best practices. He is actively seeking opportunities to apply his skills and grow within dynamic environments.

**Connect With Matías**
*   **Email:** matiasnortiz.dev@gmail.com
*   **LinkedIn:** [Link available on landing page]
*   **GitHub:** mtsortiz
*   **Download CV:** [Link available on landing page]